In [200]:
# https://canvas.vu.nl/courses/72644/files/folder/Assignments
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from scipy.interpolate import interp1d


## Part 1: IRS

In [204]:
usbond_df = pd.read_excel('data/usbond_data.xlsx')
bondtest_data = pd.read_excel('data/bondtest_data.xlsx')
european_zerorates = pd.read_excel('data/european_zerorates.xlsx')
european_zerorates = european_zerorates.set_index('Maturity')
european_zerorates =  european_zerorates['AAA rated']

In [206]:
def get_euribordata():

    url = "https://euribor.p.rapidapi.com/all"

    headers = {
        "X-RapidAPI-Key": "46bc3a6f18mshb2d07e9eb01ef29p1c2d78jsn578089492ef1",
        "X-RapidAPI-Host": "euribor.p.rapidapi.com"
    }

    EURIBOR = requests.get(url, headers=headers).json()
    
    euribor_rates = {
        '3months': EURIBOR['3months'],
        '6months': EURIBOR['6months'],
        '12months': EURIBOR['12months']
    }
    return euribor_rates

In [205]:
def calculate_cash_flows(notional, start_date, end_date, payment_frequency, fixed_rate, zero_rates):
    # Determine payment dates based on payment frequency
    payment_dates = pd.date_range(start=start_date, end=end_date, freq='6M')

    cash_flows_fixed = []  # Cash flows for fixed-rate leg
    cash_flows_floating = []  # Cash flows for floating-rate leg
    
    for i in range(len(payment_dates)):
        if i == 0:
            # Initial notional exchange on the start date
            cash_flows_fixed.append(notional)
            cash_flows_floating.append(-notional)
        else:
            # Calculate fixed-rate cash flow
            fixed_cash_flow = notional * fixed_rate * payment_frequency
            cash_flows_fixed.append(fixed_cash_flow)
            
            # Calculate floating-rate cash flow based on the zero rates
            start_date = payment_dates[i - 1]
            end_date = payment_dates[i]
            ttm = (end_date - start_date).days / 365.0  # Time to maturity
            
            # Find the corresponding zero rate from the dictionary
            zero_rate = zero_rates.get(ttm, 0.0)  # Default to 0 if not found
            floating_cash_flow = notional * zero_rate * payment_frequency
            cash_flows_floating.append(floating_cash_flow)
    
    return cash_flows_fixed, cash_flows_floating

In [208]:
def get_floating_discount_rates(payment_frequency, euribor_rates):
    # Initialize an empty list to store the floating discount rates
    floating_discount_rates = []
    
    for tenor, rate in euribor_rates.items():
        # Calculate the corresponding discount rate for each Euribor rate
        discount_rate = 1 / ((1 + rate) ** (1/payment_frequency))
        floating_discount_rates.append(discount_rate)
    
    return floating_discount_rates

def determine_discount_rate(payment_frequency, yield_curve, one_curve=True):
    if one_curve:
        zero_rates = yield_curve['zero_rate']
        if payment_frequency == 0.5:
            return zero_rates
        elif payment_frequency == 1.0:
            interpolated_rates = np.interp(np.arange(0.5, max(zero_rates.keys()) + 0.5, 0.5), list(zero_rates.keys()), zero_rates.values())
            return interpolated_rates
        else:
            raise ValueError("Unsupported payment frequency. Only 0.5 (semi-annual) and 1.0 (annual) are supported.")
    else:
        # In the two-curve scenario, you can access the Euribor data from the yield_curve dictionary
        euribor_rates = yield_curve['euribor']
        # Implement the logic to determine floating discount rates based on Euribor or other relevant curve
        # This will depend on your specific data and requirements.
        # For example:
        floating_discount_rates = get_floating_discount_rates(payment_frequency, euribor_rates)
        return yield_curve['zero_rate'], floating_discount_rates
    

def value_interest_rate_swap(zero_rate, notional, start_date, end_date, payment_frequency, fixed_rate, swap_type='fixed_vs_floating', amortization_schedule=None, one_curve = True):
    
    yield_curve = {}
    # Retrieve data: 
    # yield_curve['zero_rate'] = bootstrap_yield_curve(df, payment_frequency)

    yield_curve['zero_rate'] = zero_rate
    yield_curve['euribor'] = get_euribordata()

    if one_curve == True: 
        discount_rates = determine_discount_rate(payment_frequency, yield_curve, one_curve=True)
        fixed_cash_flows, floating_cash_flows = calculate_cash_flows(notional, start_date, end_date, payment_frequency, fixed_rate, yield_curve)
    else:
        discount_rates = determine_discount_rate(payment_frequency, yield_curve, one_curve=False)
    
    npv_value = 0
    for i, (cf, dr) in enumerate(zip(fixed_cash_flows, discount_rates)):
        npv_value += cf / ((1 + dr) ** (i + 1))    
    
    
    return npv_value


notional_amount = 100
start_date = pd.to_datetime('2023-01-01')
end_date = pd.to_datetime('2030-01-01')
payment_frequency = 0.5  # Semi-annual payments
fixed_rate = 0.03  # 3% fixed rate

npv = value_interest_rate_swap(european_zerorates, notional_amount, start_date, end_date, payment_frequency, fixed_rate)
npv   

20.99392855509883

## Bootstrap method: 
- Currently not sure which bonds to use to find zero rates

In [145]:
def bootstrap_yield_curve(df, payments_per_year):
    """"
    Bootstrap method to determine the zero curve     
    """

    zero_rates = {}

    for index, bond in df.iterrows():
        TTM = bond['Time to Maturity']
        price = bond['Bond price']
        principle = bond['Bond Principal']
        coupon_rate = bond['Coupon per year'] * payments_per_year

        if coupon_rate == 0:
            # Zero-coupon bond
            zero_rate = -np.log(price / principle) * 1 / TTM
        else:
            # Coupon-bearing bond
            total_discounted_cash_flow = []
            for t in np.arange(0.5, TTM + 0.5, 0.5):
                cash_flow = coupon_rate

                if t in zero_rates:
                    discount_factor = np.exp(-zero_rates[t] * t)
                    total_discounted_cash_flow.append(cash_flow * discount_factor)

            coupon_payment = np.sum(total_discounted_cash_flow)

            zero_rate = -np.log((price - coupon_payment) / (principle + coupon_rate)) * (1 / TTM)

        zero_rates[TTM] = zero_rate
        df.loc[index, 'zero_rate'] = zero_rate

    return zero_rates

In [210]:
bootstrap_yield_curve(bondtest_data, 0.5)

{0.5: 0.10469296074441824,
 1.0: 0.10536051565782628,
 1.5: 0.10680926388170528,
 2.0: 0.10808027549746793}